In [8]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tensorflow.keras.datasets import fashion_mnist


The breast cancer dataset is a classic and very easy binary classification dataset.


Classes: 2

Samples per class: 212(M),357(B)

Samples total: 569

Dimensionality: 30

Features: real, positive


In [9]:
from sklearn.datasets import load_breast_cancer

# Load data
data = load_breast_cancer()
X, y = data.data, data.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

y_train.sum()/y_train.shape[0], y_test.sum()/y_test.shape[0]


X_train shape: (455, 30) y_train shape: (455,)
X_test shape: (114, 30) y_test shape: (114,)


(0.6285714285714286, 0.6228070175438597)

In [10]:
def build_model(learning_rate=0.001):
    model = Sequential([
        Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [11]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(model=build_model, verbose=0)

# Define parameter grid
param_grid = {
    "model__learning_rate": [0.1, 0.01, 0.001],
    "batch_size": [16, 32],
    "epochs": [50]
}

# GridSearchCV setup
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)


2025-04-18 15:02:21.258665: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-18 15:02:21.381604: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-18 15:02:22.594963: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-18 15:02:22.836810: I tensorflow/core/platform/cpu_featu

In [12]:
print("Best Parameters:", grid_result.best_params_)
print("Best Cross-Val Score: {:.4f}".format(grid_result.best_score_))

# Evaluate on test set
from sklearn.metrics import classification_report

y_pred = (grid_result.best_estimator_.predict(X_test) > 0.5).astype("int")
print(classification_report(y_test, y_pred))


Best Parameters: {'batch_size': 32, 'epochs': 50, 'model__learning_rate': 0.01}
Best Cross-Val Score: 0.9318
              precision    recall  f1-score   support

           0       0.84      1.00      0.91        43
           1       1.00      0.89      0.94        71

    accuracy                           0.93       114
   macro avg       0.92      0.94      0.93       114
weighted avg       0.94      0.93      0.93       114



# Random Search

Absolutely! The `tuner` in `keras_tuner` is like your **smart assistant for hyperparameter tuning**. It automates the process of searching for the best set of hyperparameters for your model.

Let’s break it down clearly:

---

### 🔍 **What does the tuner do?**

A `tuner`:

1. **Receives a `build_model(hp)` function**  
   This function tells the tuner what parts of the model can change (like number of units in a layer, learning rate, etc.).

2. **Chooses combinations of hyperparameters**  
   Based on the type of tuner (e.g. `RandomSearch`, `Hyperband`, `BayesianOptimization`), it selects a set of hyperparameter values from the defined search space.

3. **Trains multiple models**  
   For each combination of hyperparameters:
   - It builds a model
   - Trains it on your training data
   - Evaluates it on validation data

4. **Tracks the best model**  
   It keeps track of:
   - The **best model (weights)**  
   - The **best combination of hyperparameters**

---

### 🛠️ **How does it work under the hood (for `RandomSearch`)?**

```text
→ You define a search space (via hp.Choice, hp.Int, etc.)
→ Tuner randomly samples hyperparameters from this space
→ Builds and trains the model using those values
→ Evaluates on validation set (e.g., using val_accuracy)
→ Repeats this for a defined number of trials (max_trials)
→ Selects the combination that gave the best result
```

So in your case with:

```python
max_trials=10
```

→ It will try **10 different combinations** of:
- `units1`: 16 to 128 (step 16)
- `units2`: 8 to 64 (step 8)
- `learning_rate`: 0.01, 0.001, 0.0001

---

### 🧠 Why is it useful?

- You don’t have to manually loop through all combos
- It helps avoid underfitting or overfitting
- Improves model performance with minimal effort
- You can plug it into **any Keras model** with minimal code change

---

In [15]:
! pip install keras-tuner
import kerastuner as kt

/var/folders/wd/_vtbthvs7szg1hl8nj3sx92w0000gn/T/ipykernel_47757/1660963963.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [16]:
# Hyperparameter tuning with Keras Tuner 
def build_model(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Int('units1', min_value=16, max_value=128, step=16),
        activation='relu',
        input_shape=(X_train.shape[1],)
    ))
    
    model.add(Dense(
        units=hp.Int('units2', min_value=8, max_value=64, step=8),
        activation='relu'
    ))
    
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    # Tune learning rate
    lr = hp.Choice('learning_rate', [0.01, 0.001, 0.0001])
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


In [17]:
# Set up RandomSearch Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Try 10 combinations
    executions_per_trial=1,
    directory='random_search_dir',
    project_name='breast_cancer_tuning'
)


/Users/sumitsontakke/Documents/Dev/PES/mlenv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
# Perform Search
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)


Trial 10 Complete [00h 00m 07s]
val_accuracy: 0.8901098966598511

Best val_accuracy So Far: 0.9340659379959106
Total elapsed time: 00h 01m 07s


🎯 Summary:
- The tuner just tries random combinations from the space you define.
- For each combo, it builds → trains → evaluates → remembers the best.
- You can access the top model and hyperparams after tuning is done.

In [20]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best learning rate:", best_hp.get('learning_rate'))
print("Best units1:", best_hp.get('units1'))
print("Best units2:", best_hp.get('units2'))


Best learning rate: 0.01
Best units1: 96
Best units2: 48


In [21]:
best_model = tuner.hypermodel.build(best_hp)
history = best_model.fit(X_train, y_train, epochs=50, validation_split=0.2)

# Evaluate on test set
loss, accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.4861 - loss: 69.6144 - val_accuracy: 0.4066 - val_loss: 4.3360
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5588 - loss: 6.7959 - val_accuracy: 0.7253 - val_loss: 1.7698
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7746 - loss: 2.0688 - val_accuracy: 0.8791 - val_loss: 0.6680
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9121 - loss: 0.5685 - val_accuracy: 0.8462 - val_loss: 0.5957
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8924 - loss: 0.6214 - val_accuracy: 0.8462 - val_loss: 0.5322
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9159 - loss: 0.4699 - val_accuracy: 0.9121 - val_loss: 0.5674
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9095 - loss: 0.4011 - val_accuracy: 0.9121 - val_loss: 0.3787
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9222 - loss: 0.3393 - val_accuracy: 0.9011 - val_los